# Bestandsabfrage von Judaica Zeitschriften 
über die DNB SRU Schnittstelle

nutzt MARC21-XML (https://www.loc.gov/marc/bibliographic/)


In [1]:
# for the sake of readability remove the warnings
import warnings

warnings.filterwarnings('ignore')

In [17]:
!pip install jewish

  Preparing metadata (setup.py) ... done
  Created wheel for jewish: filename=jewish-0.1-py3-none-any.whl size=6025 sha256=390107b3430ab27058d33dc27039a4b70438c145dd79f37d069385240844828c
  Stored in directory: /Users/benni/Library/Caches/pip/wheels/ce/cb/03/02e7238fbdb28dc17e0338768393249552314ee1e940e2fea8
Successfully built jewish

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


## Import the libraries

In [2]:
import pandas as pd
import numpy as np
import os

# Get the metadata

In [62]:
df_metadata = pd.read_excel('metadata/CM_Seiten_Metadaten.xlsx')
df_metadata.head()

## Statistics on the Metadata  

In [3]:
df_metadata.shape

(918165, 14)

In [4]:
df_metadata.sample(5)

,VLID_Seite,OT_PATH,VLID_Zs,VLID_Parent,Parent Knotentyp,Parent-Type,Zs_Caption,Volume_Caption,Heft_Caption,Aufsatz_Caption,Datum,Seite (OT_SORT),Seite_Caption,Seitenzahl_Caption
291703,2677107,|2651273||2652877||2652929||2677107|,2651273,2652929,Metadaten,journal issue,Jüdische Rundschau,8 (1835),Heft 56 (15.7.1927),NaN,1927-07-15,5,NaN,(null)
486726,3057879,|3053752||3053911||3053937||3057879|,3053752,3053937,Metadaten,journal issue,Die Stimme,8 (1835),Heft 183 (9.7.1931),NaN,1931-07-09,10,NaN,(null)
445275,3005520,|2979076||2980834||2980929||3005520|,2979076,2980929,Metadaten,journal issue,Wiener Morgenzeitung,8 (1835),Heft 1846 (5.4.1924),NaN,1924-04-05,10,Seite 10,10
38410,2290652,|2277379||2278097||2278119||2290652|,2277379,2278119,Metadaten,journal issue,Central-Verein-Zeitung,8 (1835),22 (30.5.1935),NaN,1935-05-30,10,NaN,(null)
265874,2644562,|2644300||2644301||2644562|,2644300,2644301,Metadaten,journal volume,Jahrbuch der Gesellschaft für Geschichte der J...,1 (1929),NaN,NaN,1929-01-01,220,Seite 216,216


In [5]:
# pandas get all headers
df_metadata.columns

Index(['VLID_Seite', 'OT_PATH', 'VLID_Zs', 'VLID_Parent', 'Parent Knotentyp',
       'Parent-Type', 'Zs_Caption', 'Volume_Caption', 'Heft_Caption',
       'Aufsatz_Caption', 'Datum', 'Seite (OT_SORT)', 'Seite_Caption',
       'Seitenzahl_Caption'],
      dtype='object')

## Bereinigung der Daten

In [63]:
# drop columns VLID Seite, OT_PAHT, VLID_Zs
df_metadata.drop(['VLID_Seite', 'OT_PATH', 'VLID_Zs', 'VLID_Parent', 'Parent Knotentyp', 'Seite (OT_SORT)', 'Seite_Caption', 'Seitenzahl_Caption'], axis=1, inplace=True)

In [5]:
# list all values in column 'Parent-Type'
df_metadata['Parent-Type'].unique()

array(['journal volume', 'journal', 'journal issue', 'chapter',
       'title page', 'additional', 'article', 'address', 'correspondence',
       'illustration document', 'spine', 'struct article',
       'table of contents', 'piece of music', 'dedication',
       'scan card folder', 'table', 'illustrations',
       'Material to be reviewed', 'section', 'endsheet', 'preface',
       'journal additional', 'imprint', 'back cover', 'front cover',
       'stamps', 'annotation', 'corrigenda'], dtype=object)

## Get all Journal titles   

In [8]:
df_metadata['Zs_Caption'].unique()

array(['... Bericht ueber den Verein für die Provinz Westfalen zur Bildung von Elementar-Lehrern und Befoerderung von Handwerken und Kuensten unter den Juden',
       'Führer durch die jüdische Gemeindeverwaltung und Wohlfahrtspflege in Deutschland',
       'Sechzehnter Bericht über die Religionsschule der jüdischen Gemeinde, womit zu der Sonntag den 26. März in dem Hörsaale der jüdischen Knaben-Schule ... öffentlichen Prüfung ... einladet',
       '... Bericht ueber den Verein für Westfalen und Rheinprovinz zur Bildung von Elementar-Lehrern und Befoerderung von Handwerken und Kuensten unter den Juden',
       "... Verwaltungsbericht des Haupt-Grenz-Comité's zu Königsberg i. Pr. für Beseitigung der Nothstände unter den Israeliten West-Rußlands",
       'Führer durch die jüdische Wohlfahrtspflege in Deutschland',
       'Bericht der Isr. Religionsschule zu Kiel',
       'Actes et conférences de la Société des Études Juives',
       'Jüdischer Almanach', 'Österreichisch-ungarische Cantor

In [64]:
# remove duplicates in column 'Zs_Caption'
df_metadata['Zs_Caption'] = df_metadata['Zs_Caption'].str.replace(' \(.*\)', '')

In [10]:
df_metadata['Zs_Caption'].unique()

array(['... Bericht ueber den Verein für die Provinz Westfalen zur Bildung von Elementar-Lehrern und Befoerderung von Handwerken und Kuensten unter den Juden',
       'Führer durch die jüdische Gemeindeverwaltung und Wohlfahrtspflege in Deutschland',
       'Sechzehnter Bericht über die Religionsschule der jüdischen Gemeinde, womit zu der Sonntag den 26. März in dem Hörsaale der jüdischen Knaben-Schule ... öffentlichen Prüfung ... einladet',
       '... Bericht ueber den Verein für Westfalen und Rheinprovinz zur Bildung von Elementar-Lehrern und Befoerderung von Handwerken und Kuensten unter den Juden',
       "... Verwaltungsbericht des Haupt-Grenz-Comité's zu Königsberg i. Pr. für Beseitigung der Nothstände unter den Israeliten West-Rußlands",
       'Führer durch die jüdische Wohlfahrtspflege in Deutschland',
       'Bericht der Isr. Religionsschule zu Kiel',
       'Actes et conférences de la Société des Études Juives',
       'Jüdischer Almanach', 'Österreichisch-ungarische Cantor

In [11]:
df_metadata.sample(5)

,Parent-Type,Zs_Caption,Volume_Caption,Heft_Caption,Aufsatz_Caption,Datum
400943,article,Menorah,8 (1835),Heft 12 (Dezember 1926),Telschi,1926-12-01
510832,journal issue,Die Wahrheit,8 (1835),Heft 6 (11.2.1938),NaN,1938-02-11
551586,journal issue,Allgemeine Zeitung des Judenthums,8 (1835),Heft 33 (13.7.1837),NaN,1837-07-13
900508,journal issue,Das Echo,8 (1835),205 (15.6.1959),NaN,1959-06-15
455150,journal issue,Wiener Morgenzeitung,8 (1835),Heft 2888 (13.3.1927),NaN,1927-03-13


In [7]:
df_metadata['Aufsatz_Caption'].unique()

array([nan,
       "Liste des membres de la Société des Études juives pendant l'année 1888",
       'Procès-verbaux des séances du conseil', ...,
       'Bericht über die 50. Mitgliederversammlung des jüdischen Lehrervereins für Bayern e. V. am 30. und 31. August 1931 in Ansbach',
       'Unsere Antwort an Herrn Bezirksrabbiner Dr. Breuer in Aschaffenburg',
       '1924-1925'], dtype=object)

## load the pickle file

In [ ]:
import pickle

# Open the Pickle file
with open('metadata/journal_metadata/journal_list.pickle', 'rb') as file:
    # Load the Pickle data
    loaded_data = pickle.load(file)

# Close the file
file.close()

# Now you can work with the loaded data
print(loaded_data)


## Load the JSON file

In [ ]:
import json

with open('metadata/journal_metadata/journal_metadata_title_lang.json', 'r') as file:
    data = json.load(file)

In [ ]:
import pprint
pprint.pprint(data)

# DNB SRU

In [8]:
import requests
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd


# Funktion zur DNB SRU Abfrage

In [20]:
def dnb_sru(query, library='dnb', start=1, max_records=1000):
    base_url = "https://services.dnb.de/sru/" + library
    params = {'recordSchema' : 'MARC21-xml',
              'operation': 'searchRetrieve',
              'version': '1.1',
              'maximumRecords': max_records,
              'startRecord': start,
              'query': query
              }
    r = requests.get(base_url, params=params)
    xml = soup(r.content)
    records = xml.find_all('record', {'type':'Bibliographic'})

    if len(records) < 100:
        return records

    else:
        num_results = 100
        i = 101
        while num_results == 100:
            params.update({'startRecord': i})
            r = requests.get(base_url, params=params)
            xml = soup(r.content)
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)

        return records

In [24]:
records = dnb_sru(query='tit=Judaica', library='dnb', start=1, max_records=1000)
print(len(records), 'Ergebnisse')

/Users/benni/.venv/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


1423 Ergebnisse


In [25]:
import xml.dom.minidom
temp = soup(open(records), "xml")
new_xml = temp.prettify()
print(new_xml)

TypeError: unhashable type: 'ResultSet'

# print the xml tree in the first result


In [21]:
def parse_record(record):

    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))

    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'

    # title
    title = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    subtitle = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'b']", namespaces=ns)
   
    # library
    library = xml.xpath("marc:datafield[@tag = '040']/marc:subfield[@code = 'a']", namespaces=ns)
   
    # category
    category = xml.xpath("marc:datafield[@tag = '084']/marc:subfield[@code = 'a']", namespaces=ns)
   
    # date
    date = xml.xpath("marc:datafield[@tag = '264']/marc:subfield[@code = 'c']", namespaces=ns)
   
    # person 
    person = xml.xpath("marc:datafield[@tag = '100']/marc:subfield[@code = 'a']", namespaces=ns)
   
    # ddc
    ddc = xml.xpath("marc:datafield[@tag = '082']/marc:subfield[@code= 'a']", namespaces=ns)
   
    #issn
    issn = xml.xpath("marc:datafield[@tag = '022']/marc:subfield[@code= 'a']", namespaces=ns)
   
    # language
    language = xml.xpath("marc:datafield[@tag = '041']/marc:subfield[@code= 'a']", namespaces=ns)
   
    # GND Number
    gnd_number = xml.xpath("marc:datafield[@tag = '024']/marc:subfield[@code = 'a']", namespaces=ns)
   
    # Extract Statement of Responsibility
    responsibility = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'c']", namespaces=ns)
   
    # Extract Publication Information
    publication_info = xml.xpath("marc:datafield[@tag = '260']", namespaces=ns)
   
    # Extract Frequency and Regularity of Publication
    frequency = xml.xpath("marc:datafield[@tag = '310']/marc:subfield[@code = 'a']", namespaces=ns)
   
    # Extract Subject Access Points
    subjects = xml.xpath("marc:datafield[@tag = '650']/marc:subfield[@code = 'a' or @code = 'x' or @code = 'v']", namespaces=ns)
   
    # Extract Electronic Location and Access
    electronic_access = xml.xpath("marc:datafield[@tag = '856']/marc:subfield[@code = 'u']", namespaces=ns)
    
    # Extract Physical Description
    physical_description = xml.xpath("marc:datafield[@tag = '300']/marc:subfield[@code = 'a']", namespaces=ns)
    
    # Extract Series Statement
    series_statement = xml.xpath("marc:datafield[@tag = '440']/marc:subfield[@code = 'a']", namespaces=ns)
    
    # Extract Other Standard Identifier
    other_identifier = xml.xpath("marc:datafield[@tag = '024']/marc:subfield[@code = 'a']", namespaces=ns)
    
    # Extract Notes
    notes = xml.xpath("marc:datafield[@tag = '500']/marc:subfield[@code = 'a']", namespaces=ns)
    
    # Extract Subject Headings
    subjects = xml.xpath("marc:datafield[@tag = '650']/marc:subfield[@code = 'a' or @code = 'x' or @code = 'v']", namespaces=ns)
    
    # Extract Linking Entry Fields
    linking_entry = xml.xpath("marc:datafield[starts-with(@tag, '76') or starts-with(@tag, '77') or starts-with(@tag, '78')]", namespaces=ns)
    
    # Extract Classification Numbers (Dewey Decimal Classification)
    dewey_classification = xml.xpath("marc:datafield[@tag = '082']/marc:subfield[@code = 'a']", namespaces=ns)
    
    # Extract Classification Numbers (Government Document Classification)
    gov_doc_classification = xml.xpath("marc:datafield[@tag = '086']/marc:subfield[@code = 'a']", namespaces=ns)


    try:
        title = title[0].text
        library = library
        subtitle = subtitle[0].text if len(subtitle) > 0 else ""
        category = category[0].text if len(category) > 0 else ""
        date = date[0].text if len(date) > 0 else ""
        person = person[0].text if len(person) > 0 else ""
        ddc = ddc[0].text if len(ddc) > 0 else ""
        issn = issn[0].text if len(issn) > 0 else ""
        language = language[0].text if len(language) > 0 else ""
        gnd_number = gnd_number[0].text if len(gnd_number) > 0 else ""
        responsibility = responsibility[0].text if len(responsibility) > 0 else ""
        publication_info = publication_info[0].text if len(publication_info) > 0 else ""
        frequency = frequency[0].text if len(frequency) > 0 else ""
        subjects = [subject.text for subject in subjects]
        electronic_access = electronic_access[0].text if len(electronic_access) > 0 else ""
        notes = notes[0].text if len(notes) > 0 else ""
        physical_description = physical_description[0].text if len(physical_description) > 0 else ""
        series_statement = series_statement[0].text if len(series_statement) > 0 else ""
        other_identifier = other_identifier[0].text if len(other_identifier) > 0 else ""
        linking_entry = [entry.text for entry in linking_entry]
        dewey_classification = dewey_classification[0].text if len(dewey_classification) > 0 else ""
        gov_doc_classification = gov_doc_classification[0].text if len(gov_doc_classification) > 0 else ""
        subjects = [subject.text for subject in subjects]
        
        #titel = unicodedata.normalize("NFC", titel)
    except:
        title = "unkown"

    meta_dict = {
        "idn":idn,
        "library":library,
        "title":title,
        "subtitle":subtitle,
        "category":category,
        "date":date,
        "person":person,
        'ddc':ddc,
        'issn':issn,
        'language':language,
        'gnd_number':gnd_number,
        'responsibility':responsibility,
        'publication_info':publication_info,
        'frequency':frequency,
        'subjects':subjects,
        'electronic_access':electronic_access,
        'notes':notes,
        'physical_description':physical_description,
        'series_statement':series_statement,
        'other_identifier':other_identifier,
        'linking_entry':linking_entry,
        'dewey_classification':dewey_classification,
        'gov_doc_classification':gov_doc_classification
        }

    return meta_dict

In [36]:
output = [parse_record(record) for record in records]
df_test = pd.DataFrame(output)
df_test

,idn,library,title,subtitle,category,date,person,ddc,issn,language,...,frequency,subjects,electronic_access,notes,physical_description,series_statement,other_identifier,linking_entry,dewey_classification,gov_doc_classification
0,1014351715,[[]],Dialoge über die Liebe - Dialoghi di Amore. Di...,Sophia und Philone's erster Dialog: Liebe und ...,,2025,"Ebreo, Leone",100,,ger,...,,[],,,120 Seiten,,9783930177233,[],100,
1,1269033794,[[]],Sammy Gronemann: Kritische Gesamtausgabe/Erinn...,,,2025,,290,,ger,...,,[],https://www.degruyter.com/isbn/9783110694086,,800 Seiten,,9783110694086,"[\n, \n]",290,
2,1049599942,[[]],Wissenschaft des Judentums in Europe,Comparative and Transnational Perspectives,,2025,,290,,eng,...,,[],http://deposit.dnb.de/cgi-bin/dokserv?id=46259...,,550 Seiten,,9783110338607,"[\n, \n, \n, \n, \n]",290,
3,1323924604,[[]],unkown,,,2024,"Pallitsch, Lukas",830,,ger,...,,"[Literatur, Deutsch, Rezeption, Judentum]",https://doi.org/10.1515/9783111348001,,"1 Online-Ressource, 8, 461 Seiten",,9783111348001,"[\n, \n]",830,
4,1311452540,[[]],Das Nachleben des Propheten Jeremia bei Stef...,,,2024,"Pallitsch, Lukas",290,,ger,...,,[],https://www.degruyter.com/isbn/9783111347745,,"VIII, 461 Seiten",,9783111347745,"[\n, \n]",290,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1418,552067113,[[]],unkown,Quellen und Regesten zur Geschichte der Juden ...,,,,943,,,...,,"[Juden, Juden]",,"Beitr. teilw. dt., teilw. lat., teilw. hebr. -...",,,,[],943,
1419,458633801,[[]],Westfalia Judaica,Urkunden u. Regesten z. Geschichte d. Juden in...,14a,,,,,,...,,[],,,,,,[],,
1420,110737796X,[[]],unkown,,,1903,,,,ger,...,,"[Firmengründung, Teilhaber]",,,2 Ex.,,dc14025,[\n],,
1421,109556806X,[[]],Krefeld,"Judaica, Villa Merländer und anderes, Besuch e...",,1943-2001,"Loewy, Ernst",,,ger,...,,[],,,542 Blatt und 10 Hefte = 246 Einheiten maschin...,,,[],,


In [ ]:
## try to get the data for the zdb and the title Jeshurun

In [37]:
zdb_records = dnb_sru(query='tit=Jeschurun', library='zdb', start=1, max_records=1000)
print(len(zdb_records), 'Ergebnisse')

zdb_output = [parse_record(zdb_record) for zdb_record in zdb_records]
zdb_df = pd.DataFrame(zdb_output)
zdb_df

20 Ergebnisse


/Users/benni/.venv/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


,idn,library,title,subtitle,category,date,person,ddc,issn,language,...,frequency,subjects,electronic_access,notes,physical_description,series_statement,other_identifier,linking_entry,dewey_classification,gov_doc_classification
0,010346961,[[]],Jeschurun,Taschenbuch für Schilderungen und Anklänge aus...,JUDAICA,1841-1841,,290,,ger,...,,[],,,,,,[],290,
1,013007467,[[]],Jeschurun,ein Monatsbl. für u. über Israel,,1859-1861,,290,,ger,...,,[],,,,,,[],290,
2,019914873,[[]],Jeschurun,Wochenschrift zur Förderung jüdischen Geistes ...,,1883-1888,,290,,,...,,[],,,,,,[],290,
3,015319814,[[]],Der Israelit und Jeschurun,,JUDAICA,1889-1891,,290,,ger,...,,[],,,,,,[\n],290,
4,110975597X,[[]],Jeschurun,Taschenbuch für Schilderungen und Anklänge aus...,,2016,,290,,ger,...,,[],,Reproduktion,Online-Ressource,,,[\n],290,
5,98941308X,[[]],Jeschurun,Zeitschrift für die Wissenschaft des Judenthums,,1857-1878,,050,,heb,...,,[],,,,,,[\n],050,
6,989412989,[[]],Jeschurun,Zeitschrift für die Wissenschaft des Judenthums,,1857-1878,,050,,heb,...,,[],,,,,,[\n],050,
7,1059545799,[[]],Jeschurun,Zeitschrift für die Wissenschaft des Judenthums,JUDAICA,1857-1878,,050,,heb,...,,[],https://nbn-resolving.org/urn:nbn:de:hebis:30:...,,Online-Ressource,,urn:nbn:de:hebis:30:1-128639,[\n],050,
8,1059545802,[[]],Jeschurun,Zeitschrift für die Wissenschaft des Judenthums,JUDAICA,1857-1878,,050,,heb,...,,[],https://nbn-resolving.org/urn:nbn:de:hebis:30:...,,Online-Ressource,,urn:nbn:de:hebis:30:1-128627,[\n],050,
9,1059648687,[[]],Jeschurun,ein Monatsblatt zur Förderung jüdischen Geiste...,JUDAICA,1855-1888,,070,,ger,...,,[],https://www.digitale-sammlungen.de/search?quer...,,Online-Ressource,,,[\n],070,


# Daten zusammenführen

In [42]:
# Create an empty DataFrame to store the aggregated data
df_aggregated = pd.DataFrame(columns=['idn', 'title', 'subtitle', 'date'])

# Group by 'title' and 'subtitle' and aggregate 'idn' and 'date' into lists
grouped_data = zdb_df.groupby(['title', 'subtitle']).agg({'idn': list, 'date': lambda x: x.unique().tolist() if len(x.unique()) == 1 else x.tolist()}).reset_index()

# Populate the aggregated DataFrame
df_aggregated['idn'] = grouped_data['idn']
df_aggregated['date'] = grouped_data['date']
df_aggregated['title'] = grouped_data['title']
df_aggregated['subtitle'] = grouped_data['subtitle']


# Display the aggregated DataFrame
df_aggregated


,idn,title,subtitle,date
0,"[015222101, 1164510851]",Jeschurun,"Halbmonatsschrift für Pädagogik, Homiletik und...","[1892-1899, 1892-1897]"
1,"[025272896, 011861630]",Jeschurun,Monatsschrift für Lehre und Leben im Judentum,[1914-1930]
2,[014472104],Jeschurun,Organ für d. geistigen u. sozialen Interessen ...,[[1901]-1904]
3,[1171845286],Jeschurun,Organ für die geistigen und sozialen Interesse...,[[1901-1902]]
4,"[010346961, 110975597X]",Jeschurun,Taschenbuch für Schilderungen und Anklänge aus...,"[1841-1841, 2016]"
5,[019914873],Jeschurun,Wochenschrift zur Förderung jüdischen Geistes ...,[1883-1888]
6,"[98941308X, 989412989, 1059545799, 1059545802,...",Jeschurun,Zeitschrift für die Wissenschaft des Judenthums,"[1857-1878, 1857-1878, 1857-1878, 1857-1878, 1..."
7,[013007467],Jeschurun,ein Monatsbl. für u. über Israel,[1859-1861]
8,"[1059648687, 015159620]",Jeschurun,ein Monatsblatt zur Förderung jüdischen Geiste...,[1855-1888]
9,[025272799],Jeschurun,ein Monatsblatt zur Förderung jüdischen Geiste...,[1855-1887]


### Remove the temporary pickle file

In [ ]:
# remove the temp_data.pkl file
if os.path.exists('temp_data.pkl'):
    os.remove('temp_data.pkl')

## Load the titles

In [11]:
# print Zs_Caption unique values
titles = df_metadata['Zs_Caption'].unique()

### Print the titles

In [12]:
print(titles)

['... Bericht ueber den Verein für die Provinz Westfalen zur Bildung von Elementar-Lehrern und Befoerderung von Handwerken und Kuensten unter den Juden'
 'Führer durch die jüdische Gemeindeverwaltung und Wohlfahrtspflege in Deutschland'
 'Sechzehnter Bericht über die Religionsschule der jüdischen Gemeinde, womit zu der Sonntag den 26. März in dem Hörsaale der jüdischen Knaben-Schule ... öffentlichen Prüfung ... einladet'
 '... Bericht ueber den Verein für Westfalen und Rheinprovinz zur Bildung von Elementar-Lehrern und Befoerderung von Handwerken und Kuensten unter den Juden'
 "... Verwaltungsbericht des Haupt-Grenz-Comité's zu Königsberg i. Pr. für Beseitigung der Nothstände unter den Israeliten West-Rußlands"
 'Führer durch die jüdische Wohlfahrtspflege in Deutschland'
 'Bericht der Isr. Religionsschule zu Kiel'
 'Actes et conférences de la Société des Études Juives'
 'Jüdischer Almanach' 'Österreichisch-ungarische Cantoren-Zeitung'
 'Rechenschaftsbericht des Vereins "Gumpertz\'sches

In [ ]:
# go through all the titles in the column 'Zs_Caption'
# Load an existing temporary DataFrame if it exists

try:
    temp_df = pd.read_pickle('temp_data.pkl')
except FileNotFoundError:
    temp_df = pd.DataFrame()
    
counter = 1
length = len(titles)

temp_df = pd.DataFrame()
library = 'dnb'
column_headers = ['library', 'idn',  'title', 'subtitle', 'category', 'date', 'person', 'ddc', 'issn']

for title in titles:
    # search for the title in the sru interface
    print('Counter: ' + str(counter) + ' of ' + str(length))
    # calculate the percentage of the progress
    percentage = round(counter / length * 100, 2)
    print('Progress: ' + str(percentage) + '%')
    print('Searching for: ' + title)
    records = dnb_sru('tit=' + title, library=library)
    print(len(records), 'Ergebnisse')
    
    if len(records) > 0:

        output = [parse_record(record) for record in records]
        df_records = pd.DataFrame(output)
        temp_df = pd.concat([temp_df, df_records], ignore_index=True)
    counter = counter + 1
    # Save the temporary DataFrame to a Pickle file
    temp_df.to_pickle('temp_data.pkl')

In [ ]:
import warnings

warnings.filterwarnings('ignore')
# go through all the titles in the column 'Zs_Caption'
# Load an existing temporary DataFrame if it exists

try:
    temp_df = pd.read_pickle('temp_data.pkl')
except FileNotFoundError:
    temp_df = pd.DataFrame()

counter = 1
length = len(titles)

temp_df = pd.DataFrame()
library = 'zdb'

for title in titles:
    # search for the title in the sru interface
    print('Counter: ' + str(counter) + ' of ' + str(length))
    # calculate the percentage of the progress
    percentage = round(counter / length * 100, 2)
    print('Progress: ' + str(percentage) + '%')
    print('Searching for: ' + title)
    records = dnb_sru('tit=' + title, library=library)
    print(len(records), 'Ergebnisse')

    if len(records) > 0:
        output = [parse_record(record) for record in records]
        df_records = pd.DataFrame(output)
        temp_df = pd.concat([temp_df, df_records], ignore_index=True)
    counter = counter + 1
    # Save the temporary DataFrame to a Pickle file
    temp_df.to_pickle('temp_data.pkl')

In [ ]:
large_df = pd.DataFrame()
# Load the temporary DataFrame from the Pickle file
temp_df = pd.read_pickle('temp_data.pkl')

# Append the temporary DataFrame to your large DataFrame
large_df = pd.concat([large_df, temp_df], ignore_index=True)

# remove duplicates
large_df.drop_duplicates(subset=['title', 'date'], keep='first', inplace=True)

# Save the large DataFrame to a CSV file
large_df.to_csv('final_data.csv', index=True, sep=';')

large_df.to_excel('final_data.xlsx')

# Ideen, Fragen

Daten bereinigen
Wie kann ich die Anfrage präzisieren?
1. Kategorie rausfiltern, nur Judaica
2. Exaktes Matching vom Titel - z.B. Jahresbericht.

Herangehensweise 1
Bestehenden Bestand ergänzen.
2. Neue Zeitschriften aus dem Bereich Judacia oder bestimmten 

Gesamten Judaica Bestand abfragen.
1. Zeitschriften
2. Monografien

Über OAI Schnittstelle Sammlungen abfragen


# Ontologie erstellen

Typische Ontoilogie für Zeitschriften, vollständig für unsere Datenbank
Titel
Datum von bis 





# Statistische Auswertungen

Welche Daten konnten vervollständigt werden


# ZDB (Zeitschriftendatenbank)


# Interate over the titles

In [22]:
# go through all the titles

df_records_per_title = pd.DataFrame(columns=['title', 'number_of_records'])
zdb_data = pd.DataFrame()

for title in titles:
    query = f'tit={title}'
    # fetch the number of the results
    records = dnb_sru(query=query, library='zdb', start=1, max_records=1000)
    length_records = len(records)
    print(f'Tile: {title} Results: {length_records}')
    # append to the dataframe
    df_records_per_title =  df_records_per_title.append({'title': title, 'number_of_records': length_records}, ignore_index=True)
    if 0 < length_records <= 100:
         output = [parse_record(record) for record in records]
         # append the records to the zdb_data
         zdb_data = zdb_data.append(output, ignore_index=True)
        
# save the number of records to csv
df_records_per_title.to_csv('number_records.csv')
df_records_per_title

Tile: ... Bericht ueber den Verein für die Provinz Westfalen zur Bildung von Elementar-Lehrern und Befoerderung von Handwerken und Kuensten unter den Juden Results: 0
Tile: Führer durch die jüdische Gemeindeverwaltung und Wohlfahrtspflege in Deutschland Results: 2
Tile: Sechzehnter Bericht über die Religionsschule der jüdischen Gemeinde, womit zu der Sonntag den 26. März in dem Hörsaale der jüdischen Knaben-Schule ... öffentlichen Prüfung ... einladet Results: 0
Tile: ... Bericht ueber den Verein für Westfalen und Rheinprovinz zur Bildung von Elementar-Lehrern und Befoerderung von Handwerken und Kuensten unter den Juden Results: 0
Tile: ... Verwaltungsbericht des Haupt-Grenz-Comité's zu Königsberg i. Pr. für Beseitigung der Nothstände unter den Israeliten West-Rußlands Results: 0
Tile: Führer durch die jüdische Wohlfahrtspflege in Deutschland Results: 2
Tile: Bericht der Isr. Religionsschule zu Kiel Results: 0
Tile: Actes et conférences de la Société des Études Juives Results: 2
Tile: 

,title,number_of_records
0,... Bericht ueber den Verein für die Provinz W...,0
1,Führer durch die jüdische Gemeindeverwaltung u...,2
2,Sechzehnter Bericht über die Religionsschule d...,0
3,... Bericht ueber den Verein für Westfalen und...,0
4,... Verwaltungsbericht des Haupt-Grenz-Comité'...,0
...,...,...
303,Illustrirter jüdischer Familienkalender,1
304,Jahresbericht der Jugendgruppe in Frankfurt a....,0
305,Jüdisches Gemeindeblatt für den Verband der Ku...,3
306,Aḥiasaf,2


### list titles with no values
Here we have no hits. That is fine.

In [16]:
titles_with_zero_records = df_records_per_title[df_records_per_title['number_of_records'] == 0]['title']

# Print the titles
print("Titles with 0 number_of_records:")
for title in titles_with_zero_records:
    print(title)


Titles with 0 number_of_records:
... Bericht ueber den Verein für die Provinz Westfalen zur Bildung von Elementar-Lehrern und Befoerderung von Handwerken und Kuensten unter den Juden
Sechzehnter Bericht über die Religionsschule der jüdischen Gemeinde, womit zu der Sonntag den 26. März in dem Hörsaale der jüdischen Knaben-Schule ... öffentlichen Prüfung ... einladet
... Bericht ueber den Verein für Westfalen und Rheinprovinz zur Bildung von Elementar-Lehrern und Befoerderung von Handwerken und Kuensten unter den Juden
... Verwaltungsbericht des Haupt-Grenz-Comité's zu Königsberg i. Pr. für Beseitigung der Nothstände unter den Israeliten West-Rußlands
Bericht der Isr. Religionsschule zu Kiel
Österreichisch-ungarische Cantoren-Zeitung
Rechenschaftsbericht des Vereins "Gumpertz'sches Siechenhaus" und der "Minka von Goldschmidt-Rothschild-Stiftung"
[Verein zur Pflege und Unterstützung israelitischer Kranker Bikkur Cholim, gegründet v. Dr. M. Hirsch. Bericht 4, 1895/97-14,1909]
... Jahresber

### list tiles with high values
There are to much records for the title found. This needs manual inspection.

In [32]:
df_records_per_title[df_records_per_title['number_of_records'] >= 100]


,title,number_of_records
27,Im deutschen Reich,196
30,Palästina,152
39,Nationalzeitung,144
40,Neue Zeitung,152
41,Die Stimme,182
48,Nachrichtendienst,549
73,Judaica,201
99,Die Wahrheit,141
100,Programm,10120
108,Zion,117


## XXX Search for subtitles
We will check for the subtitles in the ZDB
Es gibt keinen direkten Eintrag für die Untertitel in der ZDB.
Daher prüfen wir den vollständigen Titel (tst).

In [ ]:
df_records_per_title = pd.DataFrame(columns=['title', 'number_of_records'])
zdb_data = pd.DataFrame()

for title in titles:
    query = f'tst={title}'
    # fetch the number of the results
    records = dnb_sru(query=query, library='zdb', start=1, max_records=1000)
    length_records = len(records)
    print(f'Tile: {title} Results: {length_records}')
    # append to the dataframe
    df_records_per_title =  df_records_per_title.append({'title': title, 'number_of_records': length_records}, ignore_index=True)
    if 0 < length_records <= 100:
        output = [parse_record(record) for record in records]
        # append the records to the zdb_data
        zdb_data = zdb_data.append(output, ignore_index=True)

# save the number of records to csv
df_records_per_title.to_csv('number_records.csv')
df_records_per_title

# print the records


In [33]:
zdb_data


,idn,library,title,subtitle,category,date,person,ddc,issn,language,...,frequency,subjects,electronic_access,notes,physical_description,series_statement,other_identifier,linking_entry,dewey_classification,gov_doc_classification
0,012981613,[[]],Führer durch die jüdische Gemeindeverwaltung u...,,JUDAICA,1932-1933,,340,,ger,...,,[],,,,,,"[\n, \n]",340,
1,1188418459,[[]],Führer durch die jüdische Gemeindeverwaltung u...,,JUDAICA,1932-1933,,290,,ger,...,,[],https://nbn-resolving.org/urn:nbn:de:hebis:30:...,Reproduktion,,,urn:nbn:de:hebis:30:1-157566,"[\n, \n]",290,
0,015310019,[[]],Führer durch die jüdische Wohlfahrtspflege in ...,,JUDAICA,1928-1929,,290,,ger,...,,[],,,,,,"[\n, \n]",290,
1,1188418165,[[]],Führer durch die jüdische Wohlfahrtspflege in ...,,JUDAICA,1928-1929,,290,,ger,...,,[],https://nbn-resolving.org/urn:nbn:de:hebis:30:...,Reproduktion,,,urn:nbn:de:hebis:30:1-157558,"[\n, \n]",290,
0,010131159,[[]],Actes et conférences de la Société des Études ...,,JUDAICA,1886-1889,,290,1149-8684,,...,,[],,,,,,"[\n, \n]",290,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,010269754,[[]],unkown,,JUDAICA,1937-1938,,290,,ger,...,,[Jüdische Gemeinde],,,,,,"[\n, \n, \n]",290,
1,1197251553,[[]],Jüdisches Gemeindeblatt für den Verband der Ku...,,JUDAICA,1938-1938,,070,,ger,...,,[],https://nbn-resolving.org/urn:nbn:de:hebis:30:...,Reproduktion,,,urn:nbn:de:hebis:30:1-158058,"[\n, \n]",070,
2,989711102,[[]],unkown,,JUDAICA,1937-1937,,070,,ger,...,,[Jüdische Gemeinde],https://nbn-resolving.org/urn:nbn:de:hebis:30:...,,Online-Ressource,,urn:nbn:de:hebis:30:1-158058,[\n],070,
0,010120858,[[]],Aḥiasaf,"luaḥ-ʿam sifruti ṿe-shimushi, ʿim temunot ṿe-t...",JUDAICA,1893-1924,,290,,heb,...,,[],,,,,,[\n],290,


## List all the entries which are available in CM

In [38]:
cm_data = zdb_data[zdb_data['electronic_access'].str.contains('http://sammlungen.ub.uni-frankfurt.de/cm/')]
cm_data

,idn,library,title,subtitle,category,date,person,ddc,issn,language,...,frequency,subjects,electronic_access,notes,physical_description,series_statement,other_identifier,linking_entry,dewey_classification,gov_doc_classification
1,02526933X,[[]],Zeitschrift für die Geschichte der Juden in De...,,JUDAICA,1887-1937,,290,,ger,...,,[],http://sammlungen.ub.uni-frankfurt.de/cm/id/22...,Teilweise digitalisiert in Kooperation und aus...,Online-Ressource,,,"[\n, \n]",290,
1,025267701,[[]],Altneuland,Monatsschrift für die wirtschaftliche Erschlie...,0,1904-1906,,330,,ger,...,,[],http://sammlungen.ub.uni-frankfurt.de/cm/id/22...,Gesehen am 16.06.06,Online-Ressource,,,"[\n, \n]",330,
1,980804361,[[]],KC-Blätter,Monatsschrift der im Kartell-Convent vereinigt...,JUDAICA,1910-1933,,070,,ger,...,,[],http://sammlungen.ub.uni-frankfurt.de/cm/id/22...,Gesehen am 09.08.06,Online-Ressource,,,[\n],070,
0,027047121,[[]],Bericht der Hochschule für die Wissenschaft de...,,JUDAICA,1874-1938,,290,,ger,...,,[],http://sammlungen.ub.uni-frankfurt.de/cm/id/22...,Gesehen am 16.06.06,Online-Ressource,,,[],290,
4,979070430,[[]],Bar Kochba,Blätter für die heranwachsende jüdische Jugend,JUDAICA,1919-1921,,290,,ger,...,,[],http://sammlungen.ub.uni-frankfurt.de/cm/id/22...,Gesehen am 16.06.06,Online-Ressource,,,[\n],290,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,980804566,[[]],Kalender und Jahrbuch für Israeliten,auf das Jahr ..,JUDAICA,1842-1851,,290,,ger,...,,[],http://sammlungen.ub.uni-frankfurt.de/cm/id/26...,Gesehen am 09.08.06,Online-Ressource,,,[\n],290,
1,984069410,[[]],Jüdische allgemeine Zeitung,,JUDAICA,1934-1936,,070,,ger,...,,[],http://sammlungen.ub.uni-frankfurt.de/cm/id/41...,Gesehen am 04.05.07,,,,"[\n, \n]",070,
5,025268589,[[]],Neue jüdische Monatshefte,"Zeitschrift für Politik, Wirtschaft und Litera...",JUDAICA,1916-1920,,050,,ger,...,,[],http://sammlungen.ub.uni-frankfurt.de/cm/id/29...,Gesehen am 08.01.18,Online-Ressource,,,[\n],050,
7,025269151,[[]],Menorah,"jüdisches Familienblatt für Wissenschaft, Kuns...",JUDAICA,1923-1932,,050,,ger,...,,[],http://sammlungen.ub.uni-frankfurt.de/cm/id/29...,Gesehen am 05.09.17,Online-Ressource,,,[\n],050,


In [40]:
cm_data['title'].count()

83

In [41]:
zdb_data['title'].count()

1513

In [43]:
zdb_data['title'].count() - cm_data['title'].count()

1430

## XXX Check if the titles with no records can be found 

## Save the data to a csv file

In [44]:
zdb_data.to_csv('zdb_data.csv')

# Matching
## Criteria 1: Title or subtitle

In [46]:
df_merged = pd.merge(df_metadata, zdb_data, left_on='Zs_Caption', right_on='title', how='left')
df_merged

,Parent-Type,Zs_Caption,Volume_Caption,Heft_Caption,Aufsatz_Caption,Datum,idn,library,title,subtitle,...,frequency,subjects,electronic_access,notes,physical_description,series_statement,other_identifier,linking_entry,dewey_classification,gov_doc_classification
0,journal volume,... Bericht ueber den Verein für die Provinz W...,8 (1835),NaN,NaN,1835-01-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,journal volume,... Bericht ueber den Verein für die Provinz W...,8 (1835),NaN,NaN,1835-01-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,journal volume,... Bericht ueber den Verein für die Provinz W...,8 (1835),NaN,NaN,1835-01-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,journal volume,... Bericht ueber den Verein für die Provinz W...,8 (1835),NaN,NaN,1835-01-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,journal volume,... Bericht ueber den Verein für die Provinz W...,8 (1835),NaN,NaN,1835-01-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913306,journal volume,Séance du comité de direction du ...,1914,NaN,NaN,1914-01-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2913307,journal volume,Séance du comité de direction du ...,1914,NaN,NaN,1914-01-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2913308,journal volume,Séance du comité de direction du ...,1914,NaN,NaN,1914-01-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2913309,journal volume,Séance du comité de direction du ...,1914,NaN,NaN,1914-01-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
# find all titles from df_metadata in zdb_data
#df_merged = pd.merge(df_metadata, zdb_data, left_on='Zs_Caption', right_on='title', how='left')
# find df_metadata['Zs_Caption'] in the zdb_data['title']
merged_titles = df_metadata[df_metadata['Zs_Caption'].isin(zdb_data['title'])]
merged_titles = merged_titles.drop(columns=['Parent-Type'])
merged_titles = merged_titles.reset_index(drop=True)
merged_titles = merged_titles.drop_duplicates(subset=['Zs_Caption'], keep='first')
merged_titles

,Zs_Caption,Volume_Caption,Heft_Caption,Aufsatz_Caption,Datum
0,Führer durch die jüdische Gemeindeverwaltung u...,1932-33,NaN,NaN,1932-01-01
594,Führer durch die jüdische Wohlfahrtspflege in ...,Ausgabe April 1928,NaN,NaN,1928-01-01
1216,Actes et conférences de la Société des Études ...,8 (1835),Table des matières,NaN,1889-01-01
1683,Jüdischer Almanach,5670/1910,NaN,NaN,1910-01-01
2256,Hebraeische Bibliographie,8 (1835),Editorial,NaN,1858-01-01
...,...,...,...,...,...
541486,Gemeindeblatt für die Jüdischen Gemeinden Preu...,8 (1835),2 (1.5.1934),NaN,1934-05-01
542113,Jüdisches Gemeindeblatt für die Synagogen-Geme...,8 (1835),8 (1.8.1937),NaN,1937-08-01
542371,Illustrirter jüdischer Familienkalender,5668 = 1907/08,NaN,NaN,1907-01-01
542776,Jüdisches Gemeindeblatt für den Verband der Ku...,8 (1835),15 (1.8.1937),NaN,1937-08-01


In [55]:
# test with lists
common_values = set(titles) & set(zdb_data['title'].to_list()) or set(zdb_data['subtitle'].to_list())

# Print the common values
print("Common Values:", common_values, "\n", "Number of Common Values:", len(common_values))

Common Values: {'Adressbuch für den jüdischen Buchhandel', 'Financial report of the Palestine Zionist Executive', 'Jüdisches Gemeindeblatt und Nachrichtenblatt der Gemeindeverwaltung der Israelitischen Religionsgemeinde zu Leipzig', 'KC-Mitteilungen', 'Zeitschrift für die Wissenschaft des Judenthums', 'Blau-Weiss-Blätter', 'Zeitschrift für Demographie und Statistik der Juden', 'Razsvět', 'Jüdische Rundschau', 'Jerubbaal', 'Central-Vereins-Dienst', 'Israelitische Wochenschrift für die religiösen und socialen Interessen des Judenthums', 'Blätter der Jüdischen Buchvereinigung', 'Jahrbücher für jüdische Geschichte und Litteratur', 'Bayerische israelitische Gemeindezeitung', 'Mitteilungsblatt des Landesverbandes israelitischer Religionsgemeinden Hessens', 'Dirnfurṭer prifilegirṭe tsayṭung', 'KC-Blätter', 'Luaḥ erets Yiśraʾel', 'Zeitschrift für hebraeische Bibliographie', 'Illustrirte Monatshefte für die gesammten Interessen des Judenthums', 'Korrespondenzblatt über Auswanderungs- und Siedlu

## Criteria 2: Dates

# Unmatched titles


In [57]:
# clean dates in df_metadata

# Check if values in 'date_column' match the yyyy-mm-dd format
date_format_regex = r'^\d{4}-\d{2}-\d{2}$'
is_valid_date = df_metadata['Datum'].str.match(date_format_regex)

# Filter the DataFrame to get rows where 'date_column' contains invalid dates
invalid_dates_df = df_metadata[~is_valid_date]  # Inverting the boolean mask with ~

# Print the rows with invalid dates
print("Rows with invalid dates:")
invalid_dates_df

Rows with invalid dates:


,Parent-Type,Zs_Caption,Volume_Caption,Heft_Caption,Aufsatz_Caption,Datum
1601,title page,Actes et conférences de la Société des Études ...,8 (1835),NaN,NaN,(null)
1602,title page,Actes et conférences de la Société des Études ...,8 (1835),NaN,NaN,(null)
1603,title page,Actes et conférences de la Société des Études ...,8 (1835),NaN,NaN,(null)
1604,title page,Actes et conférences de la Société des Études ...,8 (1835),NaN,NaN,(null)
1605,title page,Actes et conférences de la Société des Études ...,8 (1835),NaN,NaN,(null)
1606,title page,Actes et conférences de la Société des Études ...,8 (1835),NaN,NaN,(null)
711262,journal issue,Blätter des Jüdischen Frauenbundes für Frauena...,8 (1835),Inhalt,NaN,(null)
722742,journal additional,Die jüdische Presse,8 (1835),53 (30.12.1904),Nr. 7,(null)
722743,journal additional,Die jüdische Presse,8 (1835),53 (30.12.1904),Nr. 7,(null)
722744,journal additional,Die jüdische Presse,8 (1835),53 (30.12.1904),Nr. 7,(null)


In [59]:
df_metadata['Datum'] = pd.to_datetime(df_metadata['Datum'], errors='coerce')

# Filter the DataFrame to get rows where 'date_column' is before '1750-01-01' or is null
filtered_df = df_metadata[(df_metadata['Datum'] < '1750-01-01') | df_metadata['Datum'].isnull()]

# Print the filtered DataFrame
print("Filtered DataFrame:")
filtered_df

In [66]:
print(df_metadata['Volume_Caption'].unique().tolist())


In [3]:
# load the cleaned data for testing
df_metadata = pd.read_csv("metadata/CM_Seiten_Metadaten_cleaned.csv")
df_metadata.head(5)

,Zs_Caption,Volume_Caption,Heft_Caption,Datum
0,... Bericht ueber den Verein für die Provinz W...,8 (1835),NaN,1835-01-01
1,... Bericht ueber den Verein für die Provinz W...,8 (1835),NaN,1835-01-01
2,... Bericht ueber den Verein für die Provinz W...,8 (1835),NaN,1835-01-01
3,... Bericht ueber den Verein für die Provinz W...,8 (1835),NaN,1835-01-01
4,... Bericht ueber den Verein für die Provinz W...,8 (1835),NaN,1835-01-01


In [10]:
# Replace "(null)" strings with pd.NaT
df_metadata['Datum'] = df_metadata['Datum'].replace('(null)', pd.NaT)

# Convert 'date' column to datetime type
df_metadata['Datum'] = pd.to_datetime(df_metadata['Datum'], errors='coerce')

# Group by 'title' and aggregate 'date' to find the minimum and maximum dates
grouped_df = df_metadata.groupby('Zs_Caption')['Datum'].agg(['min', 'max']).reset_index()

# Rename the columns to 'date_from' and 'date_to'
grouped_df.rename(columns={'min': 'date_from', 'max': 'date_to'}, inplace=True)

grouped_df['year_from'] = grouped_df['date_from'].dt.year.astype('Int64')
grouped_df['year_to'] = grouped_df['date_to'].dt.year.astype('Int64')

# Print the result
grouped_df

,Zs_Caption,date_from,date_to,year_from,year_to
0,... Bericht der Jüdischen Haushaltungsschule z...,1821-01-01,1910-01-01,1821,1910
1,... Bericht ueber den Verein für Westfalen und...,1836-01-01,1836-01-01,1836,1836
2,... Bericht ueber den Verein für die Provinz W...,1835-01-01,1835-01-01,1835,1835
3,... Jahresbericht der Israelitischen Waisenans...,1915-01-01,1917-01-01,1915,1917
4,... Jahresbericht der Jüdischen Frauenvereinig...,1855-01-01,1922-01-01,1855,1922
...,...,...,...,...,...
303,Zionist library for boys and girls,1908-01-01,1908-01-01,1908,1908
304,Zuwachsverzeichnis für die Jahre ...,1924-01-01,1934-01-01,1924,1934
305,[Verein zur Pflege und Unterstützung israeliti...,1897-01-01,1911-01-01,1897,1911
306,Österreichisch-ungarische Cantoren-Zeitung,1881-12-30,1897-12-21,1881,1897


In [12]:
print(len(df_metadata['Zs_Caption'].unique()))

308


In [13]:
print(len(df_metadata['Zs_Caption'].unique()))

308


In [ ]:
# XXX Ist in den bereinigten Daten die Spalte 'Zs_Caption' schon bereinigt von Duplikaten?
Besser ohne Duplikate checken
   

In [4]:
# Clean the Volume_Caption column
print(df_metadata['Volume_Caption'].unique().tolist())
# Damit kann man eigentlich nicht arbeiten


['8 (1835)', '1932-33', '0', '9 (1836)', 'Erstes Semester 1870', 'Ausgabe April 1928', '1928/29', '1891/1892 (1892)', '1893/1894 (1894)', '1 (1886)', '2 (1887)', '3 (1888)', '4 (1889)', '5670/1910', '1902-1903 : Teilweise veränderte Neuausgabe', 'Einundzwanzigster Rechenschaftsbericht für das Jahr 1913 (1914)', 'Zwei- und dreiundzwanzigster Rechenschaftsbericht für das Jahr 1914 und 1915 (1916)', 'Vierundzwanzigster, Fünfundzwanzigster und Sechsundzwanzigster Rechenschaftsbericht für die Jahre 1916, 1917 und 1918 (1919)', 'Vierter Bericht umfassend den Zeitraum vom 2. März 1895 bis 2. März 1897 (1897)', 'Fünfter Bericht umfassend den Zeitraum vom 2. März 1896 bis 2. März 1898 (1898)', 'Sechster Bericht [1898-1901] (1901)', 'Achter Bericht : den Zeitraum vom1. Januar bis 31. Dezember 1903 umfassend (1904)', 'Vierzehnter Bericht : den Zeitraum vom 1. Januar 1909 bis 31. Dezember 1910 umfassend (1911)', 'Siebenter Bericht : den Zeitraum vom 2. März 1901 bis 31. Dezember 1902 (1903)', '14.

In [ ]:
# Clean the Heft_Caption column
print(df_metadata['Heft_Caption'].unique().tolist())

[nan, 'Table des matières', 'Heft 1 (30.12.1881)', 'Heft 2 (7.1.1882)', 'Heft 3 (15.1.1882)', 'Heft 4 (23.1.1882)', 'Heft 5 (31.1.1882)', 'Heft 6 (8.2.1882)', 'Heft 7 (16.2.1882)', 'Heft 8 (24.2.1882)', 'Heft 9 (4.3.1882)', 'Heft 10 (12.3.1882)', 'Heft 11 (20.3.1882)', 'Heft 12 (28.3.1882)', 'Heft 13 (3.4.1882)', 'Heft 14 (13.4.1882)', 'Heft 15 (21.4.1882)', 'Heft 16 (29.4.1882)', 'Heft 17 (7.5.1882)', 'Heft 18 (15.5.1882)', 'Heft 19 (23.5.1882)', 'Heft 20 (1.6.1882)', 'Heft 21 (11.6.1882)', 'Heft 22 (19.6.1882)', 'Heft 23 (27.7.1882)', 'Heft 24 (5.7.1882)', 'Heft 25 (13.7.1882)', 'Heft 26 (21.7.1882)', 'Heft 27 (29.7.1882)', 'Heft 28 (6.8.1882)', 'Heft 29 (14.8.1882)', 'Heft 30 (22.8.1882)', 'Heft 31 (30.8.1882)', 'Heft 32 (7.9.1882)', 'Heft 33 (19.9.1882)', 'Heft 34 (27.9.1882)', 'Heft 35 (9.10.1882)', 'Heft 36 (17.10.1882)', 'Heft 37 (25.10.1882)', 'Heft 38 (2.11.1882)', 'Heft 39 (10.11.1882)', 'Heft 40 (18.11.1882)', 'Heft 41 (26.11.1882)', 'Heft 42 (4.12.1882)', 'Heft 43 (12.12.18

In [36]:
import re
import numpy as np

def extract_year(text):
    if isinstance(text, str):
        pattern_heft = r'(Heft\s)(\d{4})'  # Match 4 digits after "Heft "
        text = re.remove(pattern_heft, text)
        
        pattern = r'(?<=Heft\s)(\d{4}(?:-\d{4})?)|(?<=\()(\d{4}(?:-\d{4})?)(?=\))'  # Match 4 digits or yyyy-yyyy range after "Heft " or within parentheses
        matches = re.findall(pattern, text)
        if matches:
            return matches[-1]  # Return the last match found
    return np.nan

df_metadata['Volume_Year'] = df_metadata['Volume_Caption'].apply(extract_year)
df_metadata

AttributeError: module 're' has no attribute 'remove'

In [33]:
df_metadata['Heft_Year'] = df_metadata['Heft_Caption'].apply(extract_year)
df_metadata

,Zs_Caption,Volume_Caption,Heft_Caption,Datum,Volume_Year,Heft_Year
0,... Bericht ueber den Verein für die Provinz W...,8 (1835),NaN,1835-01-01,"(, 1835)",NaN
1,... Bericht ueber den Verein für die Provinz W...,8 (1835),NaN,1835-01-01,"(, 1835)",NaN
2,... Bericht ueber den Verein für die Provinz W...,8 (1835),NaN,1835-01-01,"(, 1835)",NaN
3,... Bericht ueber den Verein für die Provinz W...,8 (1835),NaN,1835-01-01,"(, 1835)",NaN
4,... Bericht ueber den Verein für die Provinz W...,8 (1835),NaN,1835-01-01,"(, 1835)",NaN
...,...,...,...,...,...,...
918160,Séance du comité de direction du ...,1914,NaN,1914-01-01,NaN,NaN
918161,Séance du comité de direction du ...,1914,NaN,1914-01-01,NaN,NaN
918162,Séance du comité de direction du ...,1914,NaN,1914-01-01,NaN,NaN
918163,Séance du comité de direction du ...,1914,NaN,1914-01-01,NaN,NaN


In [27]:
print(df_metadata['Volume_Year'].unique().tolist())

['1835', '1932', nan, '1836', '1870', '1928', '1891', '1893', '1886', '1887', '1888', '1889', '5670', '1902-1903', '1913', '1914', '1916', '1895', '1896', '1898-1901', '1903', '1909', '1901', '1921', '1915', '1920', '1912', '1911', '1910', '1907-1908', '1918', '1919', '1922', '1917', '1904', '1847', '1849', '1851', '1852', '1853', '1857', '1859', '1858', '1854', '1856', '1892', '1860', '1861', '1862', '1864', '1863', '1865', '1866', '1868', '1867', '1869', '1884', '1881', '1880', '1879', '1878', '1877', '1871', '1872', '1875', '1874', '1873', '1876', '1883', '1882', '1914-1915', '1885', '1890', '1894', '1897', '1898', '1899', '1900', '1902', '1905', '1906', '1907', '1908', '1924', '1925', '1926', '1927', '1929', '1930', '1931', '1936', '1938', '1915-1926', '5615', '5616', '5617', '5618', '5619', '5620', '5621', '5622', '5623', '5624', '5625', '5626', '5627', '5628', '5603', '5604', '5605', '5606', '1848', '5611', '1935', '1934', '1933', '1937', '1877-1878', '1878-1879', '1879-1880', '1

In [23]:
print(df_metadata['Volume_Year'].dtype)

float64


In [24]:
# convert the year if it is bigger than 5000 from Hebrew to gregorian
import pandas as pd
from jewish import JewishDate
# Hebcal does not install

# Convert non-missing values in the "Volume_Year" column to integers
df_metadata['Volume_Year'] = pd.to_numeric(df_metadata['Volume_Year'], errors='coerce')

def convert_to_gregorian(hebrew_year):
    # Create a JewishDate object for the Hebrew year
    jewish_date = JewishDate(year=hebrew_year, month=1, day=1)
    # Convert JewishDate object to Gregorian date
    gregorian_date = jewish_date.to_date()
    # Extract the Gregorian year
    gregorian_year = gregorian_date.year
    return gregorian_year

df_metadata['Volume_Year'].fillna(0, inplace=True)

df_metadata.loc[df_metadata['Volume_Year'] > 5000, 'Volume_Year'] = df_metadata.loc[df_metadata['Volume_Year'] > 5000, 'Volume_Year'].apply(convert_to_gregorian)

print(df_metadata)


TypeError: tuple indices must be integers or slices, not float

In [34]:
print(df_metadata['Heft_Year'].unique().tolist())

[nan, ('', '1911'), ('', '1810'), ('', '1811'), ('', '1806'), ('', '1807'), ('', '1808'), ('', '1809'), ('', '1866'), ('', '1862'), ('', '1863'), ('', '1868'), ('', '1867'), ('', '1870'), ('', '1869'), ('', '1875'), ('', '1872'), ('', '1871'), ('', '1923'), ('', '1860'), ('', '1859'), ('', '1858'), ('', '1851'), ('', '1852'), ('', '1853'), ('', '1854'), ('', '1855'), ('', '1856'), ('', '1857'), ('', '1861'), ('', '1864'), ('', '1865'), ('', '1873'), ('', '1874'), ('', '1876'), ('', '1877'), ('', '1878'), ('', '1879'), ('', '1880'), ('', '1881'), ('', '1883'), ('', '1882'), ('', '1885'), ('', '1884'), ('', '1886'), ('', '1887'), ('', '1888'), ('', '1889'), ('', '1890'), ('', '1891'), ('', '1892'), ('', '1893'), ('', '1898'), ('', '1896'), ('', '1895'), ('', '1894'), ('', '1904'), ('', '1903'), ('', '1902'), ('', '1901'), ('', '1900'), ('', '1906'), ('', '1905'), ('', '1769'), ('', '1768'), ('', '1770'), ('', '1771'), ('', '1772'), ('', '1907'), ('', '1909'), ('', '1912'), ('', '1915'), 